In [2]:
from google.colab import files
uploaded = files.upload()

Saving quora_question_pairs_rus.csv.zip to quora_question_pairs_rus.csv.zip


In [0]:
import zipfile
import io

In [0]:
zf = zipfile.ZipFile(io.BytesIO(uploaded['quora_question_pairs_rus.csv.zip']), "r")
zf.extractall()

In [0]:
import pandas as pd

In [7]:
pd.read_csv('quora_question_pairs_rus.csv')[:5]

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


# Препроцессинг

In [0]:
import csv

In [0]:
with open('quora_question_pairs_rus.csv', 'r') as f:
  reader = csv.reader(f)
  corpus = list(reader)

In [10]:
corpus.pop(0)

['', 'question1', 'question2', 'is_duplicate']

In [0]:
for doc in corpus:
  doc = doc.pop(3)

In [0]:
for doc in corpus:
  doc = doc.pop(0)

In [13]:
corpus[:5]

[['Какова история кохинор кох-и-ноор-бриллиант',
  'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'],
 ['как я могу увеличить скорость моего интернет-соединения, используя vpn',
  'как повысить скорость интернета путем взлома через dns'],
 ['почему я мысленно очень одинок, как я могу это решить',
  'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23'],
 ['которые растворяют в воде быстро сахарную соль метан и углеродный диоксид',
  'какая рыба выживет в соленой воде'],
 ['астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне',
  'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне']]

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
tokenized_corpus = []
for doc in corpus:
  for sent in doc:
    sent = sent.lower().split(' ')
    tokenized_corpus.append(sent)

In [0]:
tokenized_corp = []
for doc in tokenized_corpus:
    doc = [word for word in doc if word not in mystopwords] 
    tokenized_corp.append(doc)

In [17]:
tokenized_corp[:5]

[['какова', 'история', 'кохинор', 'кох-и-ноор-бриллиант'],
 ['произойдет,',
  'правительство',
  'индии',
  'украдет',
  'кохинор',
  'кох-и-ноор-алмаз',
  'назад'],
 ['могу',
  'увеличить',
  'скорость',
  'моего',
  'интернет-соединения,',
  'используя',
  'vpn'],
 ['повысить', 'скорость', 'интернета', 'путем', 'взлома', 'dns'],
 ['почему', 'мысленно', 'очень', 'одинок,', 'могу', 'это', 'решить']]

# TF-IDF

In [0]:
DF = {}
for i in range(len(tokenized_corp)):
    tokens = tokenized_corp[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

In [0]:
for i in DF:
  DF[i] = len(DF[i])

In [23]:
DF['расстройство']

319

In [0]:
total = [x for x in DF]

In [25]:
total[:5]

['какова', 'история', 'кохинор', 'кох-и-ноор-бриллиант', 'произойдет,']

In [0]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [0]:
from collections import Counter

In [0]:
doc = 0

tf_idf = {}

for i in range(len(tokenized_corp)):
    
    tokens = tokenized_corp[i]
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((len(tokenized_corp)+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

# TF-IDF matrix 

In [0]:
from gensim import corpora, models

In [0]:
dictionary = corpora.Dictionary(tokenized_corp)

raw_corpus = [dictionary.doc2bow(t) for t in tokenized_corp]

tfidf = models.TfidfModel(raw_corpus) 
corpus_tfidf = tfidf[raw_corpus]

In [0]:
corpus_tfidf[5]

[(28, 0.5162148479620363),
 (29, 0.4681806063521687),
 (30, 0.20807361757175244),
 (31, 0.47584837646029954),
 (32, 0.14697806424325366),
 (33, 0.3262672082182407),
 (34, 0.34139420080418736)]

# Vectorized query

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
query = ['рождественские каникулы']

In [0]:
tf_idf = TfidfVectorizer(min_df = 1)

In [0]:
X = tf_idf.fit_transform(query)

In [0]:
X.toarray()

array([[0.70710678, 0.70710678]])